# Fetch and Ingest Inventory

## 1. Setup
Ensure that the required environment variables are set.

### 1a. Load environment variables

In [ ]:
! pip install python-dotenv --quiet
from dotenv import load_dotenv
load_dotenv('../.env')

### 1b. Install pre-requisite libraries

In [ ]:
! pip install requests beautifulsoup4 markdownify urllib3 --quiet
! pip install ipykernel --quiet
! pip install ipywidgets --quiet
! pip install llama-index llama-index-vector-stores-azureaisearch --quiet
! pip install azure-search-documents --quiet
! pip install azure-data-tables --quiet

## 2. Load, Fetch and Store Inventory Items

### 2a. Fetch rows of the table & ingest the url

In [ ]:
%run -i ../utilities/storeInventory.py
%run -i ../utilities/fetchAndIngest.py

# Print the number of rows in the inventory table
print(f'The inventory table contains {get_inventory_table_count()} rows.')

# Fetch rows 100 and 110 of the table & ingest the url
for inventory in get_inventory_entities_by_page(page_size=10, page_number=10):
    print(f'Indexing {inventory['Url']} ...')
    fetch_and_ingest_url_approach_001(inventory['Url'])

### 2b. Query the index for an answer using only vector search and print results with sources.

In [ ]:
from IPython.display import Markdown 

query_engine = get_search_index("001").as_query_engine(
    similarity_top_k=3,
)

#query_engine = result.as_query_engine(similarity_top_k=3)        
response = query_engine.query("What is UAC and how do I disable it?")
display(Markdown(f"{response}"))
for source in response.source_nodes:
    print(f'Source: {source.metadata['url']} with score {source.score}')
        

### 2c. Add option for Hybric Search

In [ ]:
from IPython.display import Markdown 
from llama_index.core.vector_stores.types import VectorStoreQueryMode

query_engine = get_search_index("001").as_query_engine(
    similarity_top_k=3,
    vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID
)

#query_engine = result.as_query_engine(similarity_top_k=3)        
response = query_engine.query("What is UAC and how do I disable it?")
display(Markdown(f"{response}"))
for source in response.source_nodes:
    print(f'Source: {source.metadata['url']} with score {source.score}')